# Biomass workflow and notebook

### Workflow

In [ ]:
from airflow.operators import CompressFileSensor
from cdcol_utils import other_utils
import airflow
from airflow.models import DAG
from airflow.operators import CDColQueryOperator, CDColFromFileOperator, CDColReduceOperator
from airflow.operators.python_operator import PythonOperator
from cdcol_utils import dag_utils, queue_utils, other_utils
from airflow.utils.trigger_rule import TriggerRule

from datetime import timedelta
from pprint import pprint

_params = {'minValid': 1, 'modelos': '/web_storage/downloads/6848', 'normalized': False, 'lat': (2, 3), 'lon': (-74, -73), 'products': [{'name': 'LS7_ETM_LEDAPS', 'bands': ['swir1', 'red', 'nir', 'green', 'blue', 'swir2', 'pixel_qa']}, {'name': 'LS8_OLI_LASRC', 'bands': ['swir1', 'red', 'nir', 'green', 'blue', 'swir2', 'pixel_qa']}], 'time_ranges': [('2020-01-01', '2020-06-30')], 'execID': 'exec_6848', 'elimina_resultados_anteriores': True, 'genera_mosaico': True, 'owner': 'API-REST'}


"""
Biomass Algorithm
Templeate created by Crhsitian Segura
24-feb-2021
"""
'''
_params = {
        'minValid': 1, 
        'modelos': '/web_storage/downloads/6758', 
        'normalized': False, 
        'lat': (-2, -1),
        'lon': (-71, -70),
        'products': 
        [
            {
                'name': 'LS8_OLI_LASRC', 
                'bands': ['swir1', 'nir', 'red','swir2', 'pixel_qa']
            }
        ], 
        'time_ranges': [('2017-01-01', '2017-12-31')], 
        'execID': 'biomass_5', 
        'elimina_resultados_anteriores': True, 
        'genera_mosaico': True, 
        'owner': 'cubo_master'
        }

_params['elimina_resultados_anteriores']=False
_params['products'].append(
        {
            'name':'ALOS2_PALSAR_MOSAIC',
            'bands': ['hh','hv']
            }
        )
'''

args = {
        'owner':  _params['owner'],
        'start_date': airflow.utils.dates.days_ago(2),
        'execID': _params['execID'],
        'product': _params['products'][0]
        }

dag = DAG(
        dag_id=args["execID"],
        default_args=args,
        schedule_interval=None,
        dagrun_timeout=timedelta(minutes=120)
        )

_steps = {
        'mascara': {
            'algorithm': "mascara-landsat",
            'version': '1.0',
            'queue': queue_utils.assign_queue(
                input_type='multi_temporal',
                time_range=_params['time_ranges'][0]
                ),
            'params': {'bands': _params['products'][0]['bands']}
            },
        'reduccion': {
            #'algorithm': "joiner-reduce",
            'algorithm': "joiner",
            'version': '1.0',
            #'queue': 'airflow_xlarge',
            'queue': queue_utils.assign_queue(
                input_type='multi_temporal_unidad',
                time_range=_params['time_ranges'][0],
                unidades=len(_params['products'])
            ),
            'params': {'bands': _params['products'][0]['bands']},
            'del_prev_result': _params['elimina_resultados_anteriores'],
            },
        'medianas': {
            'algorithm': "compuesto-temporal-medianas-wf",
            'version': '1.0',
            #'queue': 'airflow_xlarge',
            'queue': queue_utils.assign_queue(
                input_type='multi_temporal_unidad',
                time_range=_params['time_ranges'][0],
                unidades=len(_params['products'])
            ),
            'params': {
                'minValid': _params['minValid'],
                'normalized':_params['normalized']
                },
            'del_prev_result': _params['elimina_resultados_anteriores'],
            },
        'mosaico': {
            'algorithm': "joiner",
            'version': '1.0',
            'queue': queue_utils.assign_queue(
                input_type='multi_area',
                lat=_params['lat'],
                lon=_params['lon']
                ),
            'params': {},
            'del_prev_result': _params['elimina_resultados_anteriores'],
            },
        'entrenamiento': {
            'algorithm': "random-forest-training",
            'version': '6.0',
            'queue': queue_utils.assign_queue(
                input_type='multi_area',
                lat=_params['lat'],
                lon=_params['lon']
                ),
            'params': {
                'bands': _params['products'][0]['bands'],
                'train_data_path': _params['modelos']
                },
            #'del_prev_result': _params['elimina_resultados_anteriores'],
            'del_prev_result': False
            },
        'clasificador': {
                'algorithm': "clasificador-generico-wf",
                'version': '5.0',
                'queue': queue_utils.assign_queue(
                    input_type='multi_area',
                    lat=_params['lat'],
                    lon=_params['lon']
                    ),
                'params': {
                    'bands': _params['products'][0]['bands'],
                    'modelos': _params['modelos']
                    },
                'del_prev_result': _params['elimina_resultados_anteriores'],
                }

        }

mascara_0 = dag_utils.queryMapByTile(
        lat=_params['lat'], 
        lon=_params['lon'],
        time_ranges=_params['time_ranges'][0],
        algorithm=_steps['mascara']['algorithm'],
        version=_steps['mascara']['version'],
        product=_params['products'][0],
        params=_steps['mascara']['params'],
        queue=_steps['mascara']['queue'],
        dag=dag,
        task_id="mascara_" + _params['products'][0]['name']
        )

if len(_params['products']) > 1:
    mascara_1 = dag_utils.queryMapByTile(
            lat=_params['lat'],
            lon=_params['lon'],
            time_ranges=_params['time_ranges'][0],
            algorithm=_steps['mascara']['algorithm'],
            version=_steps['mascara']['version'],
            product=_params['products'][1],
            params=_steps['mascara']['params'],
            queue=_steps['mascara']['queue'],
            dag=dag,
            task_id="mascara_" + _params['products'][1]['name']
            )

    reduccion = dag_utils.reduceByTile(
            mascara_0 + mascara_1,
            product=_params['products'][0],
            algorithm=_steps['reduccion']['algorithm'],
            version=_steps['reduccion']['version'],
            queue=_steps['reduccion']['queue'],
            dag=dag, task_id="joined",
            delete_partial_results=_steps['reduccion']['del_prev_result'],
            params=_steps['reduccion']['params'],
            )
else:
    reduccion = mascara_0

medianas = dag_utils.IdentityMap(
        reduccion,
        product=_params['products'][0],
        algorithm=_steps['medianas']['algorithm'],
        version=_steps['medianas']['version'],
        task_id="medianas",
        queue=_steps['medianas']['queue'],
        dag=dag,
        delete_partial_results=_steps['medianas']['del_prev_result'],
        params=_steps['medianas']['params']
        )

workflow=medianas

if queue_utils.get_tiles(_params['lat'],_params['lon'])>1:
    mosaico = dag_utils.OneReduce(
            workflow,
            task_id="mosaic",
            algorithm=_steps['mosaico']['algorithm'],
            version=_steps['mosaico']['version'],
            queue=_steps['mosaico']['queue'],
            delete_partial_results=_steps['mosaico']['del_prev_result'],
            trigger_rule=TriggerRule.NONE_FAILED,
            dag=dag
            )
    workflow=mosaico

entrenamiento = dag_utils.IdentityMap(
        workflow,
        algorithm=_steps['entrenamiento']['algorithm'],
        version=_steps['entrenamiento']['version'],
        task_id="entrenamiento",
        queue=_steps['entrenamiento']['queue'],
        dag=dag,
        delete_partial_results=_steps['entrenamiento']['del_prev_result'],
        params=_steps['entrenamiento']['params']
        )

clasificador = CDColReduceOperator(
        task_id="clasificador_generico",
        algorithm=_steps['clasificador']['algorithm'],
        version=_steps['clasificador']['version'],
        queue=_steps['clasificador']['queue'],
        dag=dag,
        lat=_params['lat'],
        lon=_params['lon'],
        params=_steps['clasificador']['params'],
        delete_partial_results=_steps['clasificador']['del_prev_result'],
        to_tiff=True
        )


entrenamiento>>clasificador
workflow>>clasificador
sensor_fin_ejecucion = CompressFileSensor(task_id='sensor_fin_ejecucion',poke_interval=60, soft_fail=True,mode='reschedule', queue='util', dag=dag) 
comprimir_resultados = PythonOperator(task_id='comprimir_resultados',provide_context=True,python_callable=other_utils.compress_results,queue='util',op_kwargs={'execID': args['execID']},dag=dag) 
sensor_fin_ejecucion >> comprimir_resultados 

# Mini-algorithms

In [ ]:
import numpy as np
print(product)
print ("Masking " + product['name'])
nodata=-9999
validValues=set()
if product['name']=="LS7_ETM_LEDAPS" or product['name'] == "LS5_TM_LEDAPS":
    validValues=[66,68,130,132]
elif product['name'] == "LS8_OLI_LASRC":
    validValues=[322, 386, 834, 898, 1346, 324, 388, 836, 900, 1348]
else:
    raise Exception("Este algoritmo sólo puede enmascarar LS7_ETM_LEDAPS, LS5_TM_LEDAPS o LS8_OLI_LASRC")

cloud_mask = np.isin(xarr0["pixel_qa"].values, validValues)
for band in product['bands']:
    print("entra a enmascarar")
    xarr0[band].values = np.where(np.logical_and(xarr0.data_vars[band] != nodata, cloud_mask), xarr0.data_vars[band], -9999)
output = xarr0

In [ ]:
import xarray as xr
import glob, os,sys

output=None
xarrs=xarrs.values()
for _xarr in xarrs:
    if (output is None):
        output = _xarr
    else:
        output=output.combine_first(_xarr)

#output=xr.auto_combine(list(xarrs))
#output=xr.open_mfdataset("/source_storage/results/compuesto_de_medianas/compuesto-temporal-medianas-wf_1.0/*.nc")
#output=xr.merge(list(xarrs))

In [ ]:
#!/usr/bin/python3
# coding=utf8
import xarray as xr
import numpy as np
print ("Compuesto temporal de medianas para " + product['name'])
print(xarr0)
nodata=-9999
medians = {}
time_axis = list(xarr0.coords.keys()).index('time')
for band in product['bands']:
    if band != 'pixel_qa':
        datos = xarr0.data_vars[band].values
        allNan = ~np.isnan(datos)

        # Comentada por Aurelio (No soporta multi unidad)
        #if normalized:
        #    m=np.nanmean(datos.reshape((datos.shape[time_axis],-1)), axis=1)
        #    st=np.nanstd(datos.reshape((datos.shape[time_axis],-1)), axis=1)
        #    datos=np.true_divide((datos-m[:,np.newaxis,np.newaxis]), st[:,np.newaxis,np.newaxis])*np.nanmean(st)+np.nanmean(m)

        if normalized:
            m=np.nanmean(datos.reshape((datos.shape[time_axis],-1)), axis=1)
            st=np.nanstd(datos.reshape((datos.shape[time_axis],-1)), axis=1)

            # Expand m and st according with the data shape
            # number of coords
            coords_num = len(list(xarr0.coords.keys()))
            l = [ x for x in range(coords_num) if x != time_axis]

            m_new = m
            st_new = st
            for axis in l:
                # If axis is 0  it is equivalent to x[np.newaxis,:]
                # If axis is 1  it is equivalent to x[:,np.newaxis]
                # And so on
                m_new = np.expand_dims(m_new,axis=axis)
                st_new = np.expand_dims(st_new,axis=axis)

            print('Time axis',time_axis)
            print('New axis',l)
            print('m',m.shape)
            print('st',st.shape)
            print('st_new',st_new.shape)
            print('m_new',m_new.shape)
            datos=np.true_divide((datos-m_new), st_new)*np.nanmean(st)+np.nanmean(m)

        medians[band] = np.nanmedian(datos, time_axis)
        medians[band][np.sum(allNan, time_axis) < minValid] = -9999
        del datos

# > **Asignación de coordenadas**
ncoords=[]
xdims =[]
xcords={}
for x in xarr0.coords:
    if(x!='time'):
        ncoords.append( ( x, xarr0.coords[x]) )
        xdims.append(x)
        xcords[x]=xarr0.coords[x]
variables ={k: xr.DataArray(v, dims=xdims,coords=ncoords) for k, v in medians.items()}
output=xr.Dataset(variables, attrs={'crs':xarr0.crs})
for x in output.coords:
    output.coords[x].attrs["units"]=xarr0.coords[x].units

In [ ]:
import os,posixpath
import re
import xarray as xr
import numpy as np
import gdal
import zipfile

import datacube

import pandas as pd

from shapely.geometry import Polygon, MultiPolygon
import json
from datacube.utils import geometry
from datacube.utils.geometry import CRS


from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_validate
from sklearn.metrics import r2_score

from joblib import dump
import rasterio.features

#parametros:
#xarr0: Mosaico del compuesto de medianas
#bands: Las bandas a utilizar
#train_data_path: Ubicación de los shape files .shp

time_ranges = [("2017-01-01", "2017-12-31")] #Una lista de tuplas, cada tupla representa un periodo **** mirar de donde viene


'''
	Code edited by Crhistian Segura
 	Date: 11-Jan-2021
	Modif: Created for biomass algorithm
'''

import pandas as pd


def geometry_mask(geoms, geobox, all_touched=False, invert=False):
    """
    Create a mask from shapes.
    By default, mask is intended for use as a
    numpy mask, where pixels that overlap shapes are False.
    :param list[Geometry] geoms: geometries to be rasterized
    :param datacube.utils.GeoBox geobox:
    :param bool all_touched: If True, all pixels touched by geometries will be burned in. If
                             false, only pixels whose center is within the polygon or that
                             are selected by Bresenham's line algorithm will be burned in.
    :param bool invert: If True, mask will be True for pixels that overlap shapes.
    """
    return rasterio.features.geometry_mask([geom.to_crs(geobox.crs) for geom in geoms],
            out_shape=geobox.shape,
            transform=geobox.affine,
            all_touched=all_touched,
            invert=invert)

# Load csv files
train_zip_file_name  = train_data_path

Conglomerados = pd.read_csv(train_data_path+'/conglomerados.csv',delimiter=',')
#data_set_PPM = pd.read_csv('data_set.csv',delimiter=',')

REFdata = Conglomerados
REFdata['random1'] = np.random.randint(1,11,Conglomerados.shape[0])


## local variables
# Number of folds (k) to run (Kmax should be equal tot he number of k-folds of the training dataset, but you can run less kfolds for debugging purposes)
Kmax = 10

# Point at the attributes names of your shapefile
DV = 'Cha_HD' #Dependent variable name (column with you variable to predict)
#Variable (column) with the folds (if k=10, then the values should be from 1 to 10). Two options
#- 'kfold'-> kfold will be randomly sampled 
#- 'k10' -> kfold will use a prepared stratified random sampling (shapefile attribute) 

kfold = 'k10'

#Variable that count the number of subplot within the cluster
countCluster = 'Count'  
Nplots = 5; #Number of subplots

#Set a maximum value for your dependent variable
maxValueplots = 500

train_kfold = REFdata[REFdata['1ha']==1]
train_kfold = train_kfold[train_kfold[countCluster]>= Nplots]
train_kfold = train_kfold[train_kfold[DV] < maxValueplots]
print(f'kfold dataset: {train_kfold.shape}');


## Generacion de datos de entrenamiento a partir de los conglomerados
salidas=[]
training_labels_all=[]
training_samples_all=np.array([], dtype=np.int64).reshape(0,7)
for i in range(len(train_kfold.Cha_HD)):
    print(f'Running conglom {i+1}')
    try:
        a = json.loads(train_kfold.iloc[i]['.geo'])

        geom = geometry.Geometry(a,crs=CRS('EPSG:4326'))

        dc = datacube.Datacube(app="Cana")
        """
        ALOS = dc.load(
            product='ALOS2_PALSAR_MOSAIC',
            geopolygon=geom,
        )

        ALOS=ALOS.isel(time=0)
        ALOS
        """
        #for i in range(30):
        xarr_0 = dc.load(
            product='LS7_ETM_LEDAPS_MOSAIC',
            time=("2016-01-01", "2016-12-31"),
            geopolygon=geom,
        )

        
        mask = geometry_mask([geom], xarr_0.geobox, invert=True)
        
        xarr_0 = xarr_0.where(mask)
        xarr_0=xarr_0.isel(time=0)


        '''
        # mascara de nubes
        import numpy as np

        nodata=-9999
	    
        validValues=set()
        if product['name']=="LS7_ETM_LEDAPS" or product['name'] == "LS5_TM_LEDAPS":
            validValues=[66,68,130,132]
        elif product['name'] == "LS8_OLI_LASRC":
            validValues=[322, 386, 834, 898, 1346, 324, 388, 836, 900, 1348]
        else:
            raise Exception("Este algoritmo solo puede enmascarar LS7_ETM_LEDAPS, LS5_TM_LEDAPS o LS8_OLI_LASRC")

        cloud_mask = np.isin(xarr_0["pixel_qa"].values, validValues)
        for band in product['bands']:
            xarr_0[band].values = np.where(np.logical_and(xarr_0.data_vars[band] != nodata, cloud_mask), xarr_0.data_vars[band], np.nan)
        xarr_mask = xarr_0

        # comppuesto de medianas
        normalized = True
        minValid = 1

        medians={} 
        for band in product['bands']:
             if band != 'pixel_qa':
                datos=xarr_mask[band].values
                allNan=~np.isnan(datos) #Una mascara que indica qué datos son o no nan. 
                if normalized: #Normalizar, si es necesario.
                    #Para cada momento en el tiempo obtener el promedio y la desviación estándar de los valores de reflectancia
                    m=np.nanmean(datos.reshape((datos.shape[0],-1)), axis=1)
                    st=np.nanstd(datos.reshape((datos.shape[0],-1)), axis=1)
                    # usar ((x-x̄)/st) para llevar la distribución a media 0 y desviación estándar 1,
                    # y luego hacer un cambio de espacio para la nueva desviación y media. 
                    datos=np.true_divide((datos-m[:,np.newaxis,np.newaxis]), st[:,np.newaxis,np.newaxis])*np.nanmean(st)+np.nanmean(m)
                #Calcular la mediana en la dimensión de tiempo 
                medians[band]=np.nanmedian(datos,0) 
                #Eliminar los valores que no cumplen con el número mínimo de pixeles válidos dado. 
                medians[band][np.sum(allNan,0)<minValid]=np.nan

        del datos
        '''
        medians={} 
        # normalizar bandas
        medians["red"]   = xarr_0["red"]  /10000
        medians["nir"]   = xarr_0["nir"]  /10000
        medians["swir1"] = xarr_0["swir1"]/10000
        medians["swir2"] = xarr_0["swir2"]/10000
       

        # Calculo de indices
        medians["ndvi"]=(medians["nir"]-medians["red"])/(medians["nir"]+medians["red"])
        medians["nbr"] =(medians["nir"]-medians["swir2"])/(medians["nir"]+medians["swir2"])
        #medians["nbr2"]=(medians["swir1"]-medians["swir2"])/(medians["swir1"]+medians["swir2"])
        #medians["ndmi"]=(medians["nir"]-medians["swir1"])/(medians["nir"]+medians["swir1"])
        medians["savi"]=((medians["nir"]-medians["red"])/(medians["nir"]+medians["red"] + (0.5)))*(1.5)

        # **Asignacion de coordenadas**
        ncoords=[]
        xdims =[]
        xcords={}
        for x in xarr_0.coords:
            if(x!='time'):
                ncoords.append( ( x, xarr_0.coords[x]) )
                xdims.append(x)
                xcords[x]=xarr_0.coords[x]
        variables ={k: xr.DataArray(v, dims=xdims,coords=ncoords) for k, v in medians.items()}
        output0=xr.Dataset(variables, attrs={'crs':xarr_0.crs})
        for x in output0.coords:
            output0.coords[x].attrs["units"]=xarr_0.coords[x].units
        output0
        """
        # Calbration factor
        # Convertir a .astype(np.int32) para evitar truncamiento
        ALOS32 = ALOS.astype(np.int32)
        # https://www.eorc.jaxa.jp/ALOS-2/en/calval/calval_index.htm
        ALOS_cal = (np.log10(ALOS32*ALOS32)*10)-83
        ALOS_2 = 10**(0.1*ALOS_cal)
        rfid = (ALOS_2.hh-ALOS_2.hv)/(ALOS_2.hh+ALOS_2.hv)
        cpR = ALOS_2.hv/ALOS_2.hh

        ALOS_2
        

        #output0['hh'] = ALOS_2.hh
        #output0['hv'] = ALOS_2.hv
        #output0['rfid']=rfid
        #output0['cpR'] =cpR
        """
        
        output0=output0.where(mask)
        
        labeled_pixels=mask*1
                
        is_train = np.nonzero(labeled_pixels)
        training_labels = labeled_pixels[is_train]
        

        bands_data=[]
        for band in output0.data_vars.keys():
            # pixel_qa is removed from xarr0 by Compuesto Temporal de Medianas
            if band != 'pixel_qa':
                bands_data.append(output0[band])
        bands_data = np.dstack(bands_data)

        # limpiar bands_data[is_train] de nan
        
        training_samples_all = np.concatenate((training_samples_all, bands_data[is_train]))
        training_labels_all= np.concatenate((training_labels_all,training_labels*train_kfold.iloc[i].Cha_HD))
        
        salidas.append(output0)
        
    except Exception as e:
        print('Failed: '+ str(e))
        

print(f'training_samples: {training_samples_all.shape}')
print(f'training_labels: {training_labels_all.shape}')

training_pd_samples=pd.DataFrame(training_samples_all)
training_pd_labels=pd.DataFrame(training_labels_all, columns={DV})

nullData=training_pd_samples[0].isnull()

all_data=pd.concat((training_pd_samples[~nullData],training_pd_labels[~nullData]),axis=1)

all_data_shuf=all_data.sample(frac=1).reset_index(drop=True)
all_data_shuf=all_data_shuf.dropna()
# Select data for clasification
X=all_data_shuf.loc[:,all_data_shuf.columns!=DV]
y=all_data_shuf[DV].to_numpy()

print(f'X shape:{X.shape}')
print(f'y shape:{y.shape}')


# Estimador con los mismos parametros de gee
clf = RandomForestRegressor(n_estimators = 500,
                            max_features = 'sqrt',
                            min_samples_leaf = 1,
                            #bootstrap = True,
                            #max_samples=0.5,
                            max_leaf_nodes = None,
                            random_state = 123)

# K-Fold para la generacion de 10 modelos de RF ( entrenamiento )
cv_results = cross_validate(clf, X, y, cv = 10,
                            scoring = ('r2'),
                            return_estimator=True,
                            return_train_score=True)

# imprim los r2 de los 10 modelos con la data de entrenamiento
for i in range(10):
    ya = cv_results['estimator'][i].predict(X)
    print(f'modelo {i} r2: {r2_score(y, ya):0.4f}')

# Export models to folder
import os

if not os.path.exists(posixpath.join(folder,'models')):
    os.makedirs(posixpath.join(folder,'models'))

from joblib import dump, load
for i in range(10):
    dump(cv_results['estimator'][i], posixpath.join(folder,'models/modelo_'+str(i)+'.joblib'))




"""
# The trainning data must be in a zip folder.
train_zip_file_name  = [file_name for file_name in os.listdir(train_data_path) if file_name.endswith('.zip')][0]
train_zip_file_path = os.path.join(train_data_path,train_zip_file_name)
train_folder_path = train_zip_file_path.replace('.zip','')

print('train_zip_file_path',train_zip_file_path)
print('train_folder_path',train_folder_path)

zip_file = zipfile.ZipFile(train_zip_file_path)
zip_file.extractall(train_data_path)
zip_file.close()

#files = [f for f in os.listdir(train_data_path) if f.endswith('.shp')]
files = [f for f in os.listdir(train_folder_path) if f.endswith('.shp')]
classes = [f.split('.')[0] for f in files]
print(classes)
#shapefiles = [os.path.join(train_data_path, f) for f in files if f.endswith('.shp')]
shapefiles = [os.path.join(train_folder_path, f) for f in files if f.endswith('.shp')]

print('shapefile',shapefiles)

labeled_pixels = rasterizar_entrenamiento(shapefiles, rows, cols, geo_transform, proj)


classifier = RandomForestClassifier(n_jobs=-1, n_estimators=500)

print('trainning samples',X_train)
print('trainning labels',y_train)
#classifier = RandomForestClassifier(n_jobs=-1, n_estimators=50, verbose=1)
classifier.fit(X_train, y_train)

# Calculo de y_pred
print('Estimar y con datos de entrada')
y_pred = classifier.predict(X_test)

# Calculo de matrix de confusion
from sklearn.metrics import confusion_matrix, cohen_kappa_score, precision_score

mconf = confusion_matrix(y_test,y_pred)
# Calculo de kappa score
kappa = cohen_kappa_score(y_test, y_pred)
# Calculo de precision score
prec = precision_score(y_test, y_pred,average = 'weighted')

# Save metrics to file
with open(posixpath.join(folder+'metrics.txt'),'w') as file_metrics:
    print(f'matriz de confusion: {mconf}')
    print(f'kappa score: {kappa}')
    print(f'precision score (weighted): {prec}')
    file_metrics.write('matriz de confusion: \n'+str(mconf))
    file_metrics.write('\nkappa score: '+str(kappa))
    file_metrics.write('\nprecision score (weighted): '+str(prec))

# write shapefiles list
file = open(folder+"shapefiles_list.txt", "w")
file.write("shapefiles list = " + "\n".join(shapefiles))
file.close()



outputxcom=posixpath.join(folder,'modelo_random_forest.pkl')
with open(outputxcom, 'wb') as fid:
    print('output',classifier)
    joblib.dump(classifier, fid)

print(classifier)
"""

In [ ]:
# In[7]:
import xarray as xr
import numpy as np
from joblib import load
import warnings

import os, posixpath

# Preprocesar:
xarrs=list(xarrs.values())
print(type(xarrs))
output0 = xarrs[0]
print(type(output0))
print(f'len xarrs: {len(xarrs)}')
print(f'xarrs: {xarrs}')


print("medianas")
print(f'bandas: {list(output0.data_vars.keys())}')
print(output0)
bands_data=[]


"""
  Revisar las medianas parecen tener inconvenientes, el dato que retorna tiene mas de un slide de tiempo

"""

# Escalar bandas

output0["red"]=output0["red"]/10000
output0["nir"]=output0["nir"]/10000
output0["swir1"]=output0["swir1"]/10000
output0["swir2"]=output0["swir2"]/10000

# Agregar Indices

output0["ndvi"]=(output0["nir"]-output0["red"])/(output0["nir"]+output0["red"])
output0["nbr"] =(output0["nir"]-output0["swir2"])/(output0["nir"]+output0["swir2"])
output0["savi"]=((output0["nir"]-output0["red"])/(output0["nir"]+output0["red"] + (0.5)))*(1+0.5)

print(f'Data + indices: {output0}')


for band in output0.data_vars.keys():
    # pixel_qa is removed from xarr0 by Compuesto Temporal de Medianas
    if band != 'pixel_qa':
        bands_data.append(output0[band])
    print(f'band processed: {band}')

bands_data = np.dstack(bands_data)


rows, cols, n_bands = bands_data.shape

n_samples = rows*cols
flat_pixels = bands_data.reshape((n_samples, n_bands))

where_are_NaNs = np.isnan(flat_pixels)
print(f'NaNs in data : {where_are_NaNs.sum()}')
flat_pixels[where_are_NaNs] = -9999

where_are_Infs = np.isinf(flat_pixels)
print(f'Infs in data : {where_are_Infs .sum()}')
flat_pixels[where_are_Infs] = -9999


print(f'Flat_pixels shape: {flat_pixels.shape}')
print(f'Flat_pixels: {flat_pixels}')

# Cargar modelos

# Clasificacion de los 10 mapas
y_maps=[]

print(os.path.dirname(folder))
print('Cargar los modelos')
for i in range(10):
    clf_load = load(posixpath.join(os.path.dirname(os.path.dirname(folder)),'random-forest-training_6.0/models/modelo_'+str(i)+'.joblib'))
    print(f'Cargado modelo {i}')
    y_maps.append(clf_load.predict(flat_pixels))

# Generar 10 imagenes
kClasificaciones=[]
for i in range(10):
    y_maps[i][where_are_NaNs[:,0]]=np.nan
    classification = y_maps[i].reshape((rows, cols))
    kClasificaciones.append(classification)


# union de mapas con la mediana

result = np.apply_along_axis(np.mean,0,kClasificaciones)

print('result classification',result)


coordenadas = []
dimensiones = []
xcords = {}
for coordenada in xarrs[0].coords:
    if (coordenada != 'time'):
        coordenadas.append((coordenada, xarrs[0].coords[coordenada]))
        dimensiones.append(coordenada)
        xcords[coordenada] = xarrs[0].coords[coordenada]

valores = {"classified": xr.DataArray(result, dims=dimensiones, coords=coordenadas)}
#array = xr.DataArray(result, dims=dimensiones, coords=coordenadas)
#array.astype('float32')
#valores = {"classified": array}
print('creacion mapa biomasa')


biomasa = xr.Dataset(valores, attrs={'crs': xarrs[0].crs})
for coordenada in output0.coords:
    biomasa.coords[coordenada].attrs["units"] = xarrs[0].coords[coordenada].units
print('creacion mapa carbono')

carbono = biomasa.copy()*0.47

print('preparacion salidas')
outputs = {'biomasa': biomasa,
'carbono': carbono }
print(f'outputs:{outputs}')


classified = biomasa.classified
classified.values = classified.values.astype('float32')